In [221]:
import pandas as pd
import numpy as np
import csv
import datetime as dt
from datetime import timedelta
from datetime import datetime
import time
import matplotlib as plt
import statistics
import math
import plotly.plotly as py
from datetime import date


In [222]:
data_start = '2019-4-15'
data_end = '2019-4-30'

res = (datetime.strptime(data_end, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
adr = pd.read_csv('analytics_data_report.csv')
df = pd.DataFrame(adr)
df.columns = df.columns.str.replace(' ', '_').str.lower()
t1 = pd.to_datetime(df['referral_received'])
t2 = pd.to_datetime(df['loan_booked'])
mask = (t1 <= res) & (t1 >= data_start)
conv = df.loc[mask]
bookcount = conv.groupby('partner')['loan_booked'].count()
leadcount = conv.groupby('partner')['referral_received'].count()
rev1 = conv.groupby('partner')['gap_gross_revenue'].mean
conversions = bookcount / leadcount
conv['difference'] = t2 - t1
conv['difference'] = conv['difference'] / dt.timedelta(days=1)
df_dates = conv[['partner', 'referral_received', 'loan_booked', 'difference']]
mean_diff = df_dates.groupby('partner')['difference'].mean()
med_diff = df_dates.groupby('partner')['difference'].median()
df_summ = pd.DataFrame({'Volume': leadcount,
                       'Booked/App': conversions,
                       'Mean Time to Fund': mean_diff,
                       'Median Time to Fund': med_diff})
result = pd.merge(df, df_summ, on='partner')
result = result[np.isfinite(result['Median Time to Fund'])]
result = result[np.isfinite(result['Mean Time to Fund'])]
start = pd.to_datetime(result['referral_received'])
result['p_fund_median'] = start + result['Median Time to Fund'].map(dt.timedelta)
result['p_fund_mean'] = start + result['Mean Time to Fund'].map(dt.timedelta)
t3 = pd.to_datetime(result['p_fund_median'])
df_summ.dropna()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,Volume,Booked/App,Mean Time to Fund,Median Time to Fund
partner,,,,
AVIS,94,0.372340,16.891290,16.236111
CarGurus - Avis,10,0.300000,15.718287,14.188194
Chrysler Capital Lease,79,0.316456,18.851639,19.615972
Credit Karma,681,0.080764,23.770278,22.403472
Donlen,17,0.647059,15.219823,13.629167
Element,36,0.222222,24.168576,20.921875
Emkay,7,0.714286,23.409583,17.468056
Enterprise Fleet Management,77,0.766234,19.986323,17.904861
Fiserv GM Lease,74,0.351351,23.053632,21.586806


In [47]:
start_date = '2019-6-1'
end_date = '2019-6-16'

res2 = (datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
pred_median = (result['p_fund_median'] <= res2) & (result['p_fund_median'] >= start_date)
new_median = result.loc[pred_median]
appcount_median = new_median.groupby('partner')['referral_received'].count()
predicted_median = appcount_median * conversions
Total_median = predicted_median.sum()
pred_mean = (result['p_fund_mean'] < res2) & (result['p_fund_mean'] >= start_date)
new_mean = result.loc[pred_mean]
appcount_mean = new_mean.groupby('partner')['referral_received'].count()
predicted_mean = appcount_mean * conversions
Total_mean = predicted_mean.sum()
booked = pd.to_datetime(result['loan_booked'])
actual = (booked <= res2) & (booked >= start_date)
actual2 = result.loc[actual]
true_actual = actual2.groupby('partner')['loan_booked'].count()
prediction_test = pd.DataFrame({'Predicted Median': round(predicted_median,0),
                           'Predicted Mean': round(predicted_mean,0),
                           'Actual': true_actual})
prediction_test.loc['Total'] = prediction_test.sum()
prediction_test.dropna()

,Predicted Median,Predicted Mean,Actual
AVIS,35.0,36.0,38.0
Chrysler Capital Lease,23.0,24.0,17.0
Credit Karma,49.0,50.0,49.0
Donlen,4.0,5.0,3.0
Element,5.0,6.0,11.0
Emkay,2.0,3.0,4.0
Enterprise Fleet Management,70.0,57.0,68.0
Fiserv GM Lease,33.0,36.0,31.0
HCA - Hyundai Kia,28.0,24.0,16.0
IFS,11.0,12.0,21.0


In [223]:
cols = list(result.columns)
cols = [cols[0]] + cols[0:]
result = result[cols]
result.to_dict()
result


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning:

DataFrame columns are not unique, some columns will be omitted.



,app_id,app_id,partner,fa_tier,fa,loan_processor,funding_clerk,title_clerk,lender,state,...,gap_gross_revenue,vsc_gross_revenue,tw_gross_revenue,total_gross_revenue,Volume,Booked/App,Mean Time to Fund,Median Time to Fund,p_fund_median,p_fund_mean
0,A18068303,A18068303,Credit Karma,3,brawson,NaN,NaN,NaN,NaN,TX,...,0.0,0.0,0,0.0,681,0.080764,23.770278,22.403472,2018-07-10 09:50:00,2018-07-11 18:38:12
1,A18068304,A18068304,Credit Karma,1,jbarnes,NaN,NaN,NaN,Fort Bragg Federal Credit Unio,TX,...,571.0,0.0,0,870.0,681,0.080764,23.770278,22.403472,2018-07-10 10:01:00,2018-07-11 18:49:12
2,A18068305,A18068305,Credit Karma,1,jgorrell,NaN,NaN,NaN,NaN,AR,...,0.0,0.0,0,0.0,681,0.080764,23.770278,22.403472,2018-07-10 10:17:00,2018-07-11 19:05:12
3,A18068306,A18068306,Credit Karma,3,jrodriguez,NaN,NaN,NaN,NaN,CA,...,0.0,0.0,0,0.0,681,0.080764,23.770278,22.403472,2018-07-10 10:20:00,2018-07-11 19:08:12
4,A18068307,A18068307,Credit Karma,1,ldukes,NaN,NaN,NaN,NaN,NV,...,0.0,0.0,0,0.0,681,0.080764,23.770278,22.403472,2018-07-10 10:23:00,2018-07-11 19:11:12
5,A18068308,A18068308,Credit Karma,3,mandrada,NaN,NaN,NaN,NaN,CA,...,0.0,0.0,0,0.0,681,0.080764,23.770278,22.403472,2018-07-10 10:23:00,2018-07-11 19:11:12
6,A18068309,A18068309,Credit Karma,1,mdang_fa,NaN,NaN,NaN,NaN,UT,...,0.0,0.0,0,0.0,681,0.080764,23.770278,22.403472,2018-07-10 10:25:00,2018-07-11 19:13:12
7,A18068311,A18068311,Credit Karma,2,munnasch,NaN,NaN,NaN,Wells Fargo/IFS Direct Fund,CA,...,505.0,0.0,0,804.0,681,0.080764,23.770278,22.403472,2018-07-10 10:46:00,2018-07-11 19:34:12
8,A18068312,A18068312,Credit Karma,1,rlevy,NaN,NaN,NaN,Carter Federal Credit Union,CA,...,0.0,0.0,327,626.0,681,0.080764,23.770278,22.403472,2018-07-10 11:00:00,2018-07-11 19:48:12
9,A18068313,A18068313,Credit Karma,0,spurcell,NaN,NaN,NaN,NaN,KY,...,0.0,0.0,0,0.0,681,0.080764,23.770278,22.403472,2018-07-10 11:01:00,2018-07-11 19:49:12


In [225]:
df2 = df[['partner','gap_gross_revenue','vsc_gross_revenue','tw_gross_revenue','total_gross_revenue']]
df2 = df2.replace(0, np.NaN)

In [226]:
grouped = df2.groupby('partner').agg(np.mean)
grouped.dropna()


,gap_gross_revenue,vsc_gross_revenue,tw_gross_revenue,total_gross_revenue
partner,,,,
AVIS,456.524063,1174.787611,320.654762,1214.327029
Avis Online App,471.744561,1827.088235,324.250000,1066.155712
Avis Rental,477.568000,1270.250000,333.083333,1407.695556
CarGurus - Avis,476.487241,1316.607143,327.000000,1169.643986
Chrysler Capital Lease,491.226317,1023.567987,330.655914,1214.638358
ConsumersAdvocate,513.333333,1000.000000,377.000000,1157.800000
Credit Karma,527.489377,980.965398,318.434899,666.912462
Donlen,473.004444,1125.580247,332.971429,1187.241900
Element,471.322959,1073.359375,330.924528,1067.009121
